---

### **Import SK packages**

In [1]:
#r "nuget:Microsoft.SemanticKernel, 0.16.230615.1-preview"
//#r "nuget:Microsoft.SemanticKernel, 0.17.230718.1-preview"



Installed Packages Microsoft.SemanticKernel, 0.16.230615.1-preview

---

### **Load secrets into local variables**

In [2]:
#!import ../Settings/AppConfig.cs

//#!import /Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeployDavinci = apiConfig.deployDavinci;
string mydeployEmbed = apiConfig.deployEmbed; 
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;
string myQdrantEndpoint = apiConfig.QdrantEndpoint;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeployDavinci + " & " + mydeployEmbed + " & " + myQdrantEndpoint + " & " + myendpoint + " & " + myAPIKey);

---
### **Build the Semantic Kernel client**

In [3]:
using Microsoft.SemanticKernel;
//using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
//using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

kernel.Config.AddAzureTextCompletionService(mydeployDavinci, myendpoint, myAPIKey);

---
### **Load Skills and setup test input**

In [4]:
using System.Text.Json;
using System.Text.Json.Serialization;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

// The default input variable 
var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

Console.WriteLine(input);


Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.



---
### **Example using Stanford University's "Design Thinking"**
#### Empathize, Define, Ideate, Prototype, Test.
<img src="designthinking.png" width="50%" height="50%">

#### Let's run the "Empathize" step to understand our customers' painpoints.



In [5]:
var empathyResult = await kernel.RunAsync(input, skillDT["Empathize"]);

Console.WriteLine(empathyResult);



[
    { "sentiment": "Frustrated", "summary": "Poor customer service"},
    { "sentiment": "Angry", "summary": "Products not working as expected"},
    { "sentiment": "Dissatisfied", "summary": "Products not working despite being under warranty"}
]


---
#### Turn the empathy insights into problem definitions with the "Define" step

In [6]:
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var defineResult = await kernel.RunAsync(empathyResult.ToString(), skillDT["Define"]);

Console.WriteLine(defineResult);



[
    { "title": "Poor customer service", "problem": "Customers are not receiving satisfactory customer service from the company", "source":"Lack of training or resources for customer service staff"},
    { "title": "Product malfunction", "problem": "Products are not working as expected and are causing frustration for customers", "source":"Poor quality control during manufacturing"},
    { "title": "Product not under warranty", "problem": "Products are not covered by the company's warranty, leading to customers having to pay for repairs", "source":"Inadequate product warranties"},
    { "title": "Inadequate product support", "problem": "Customers are not receiving adequate product support from the company", "source":"Lack of resources or staff for product support"},
    { "title": "Inadequate communication", "problem": "Customers are not receiving timely or accurate communication from the company", "source":"Lack of resources or staff for customer communication"}
]


---
#### Brainstorm ideas to address the problems using the "Ideate" step

In [7]:
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var ideateResult = await kernel.RunAsync(defineResult.ToString(), skillDT["Ideate"]);

Console.WriteLine(ideateResult);
//display(ideateResult.ToString());



| Lower-hanging fruit | Higher-hanging fruit |
| :--- | :--- |
| 1. Increase customer service training for staff | 1. Develop an AI-based customer service chatbot |
| 2. Introduce a customer satisfaction survey | 2. Develop a mobile app for customers to access product support |
| 3. Implement a customer loyalty program | 3. Implement a product recall system |
| 4. Introduce a customer feedback system | 4. Introduce a warranty registration system |
| 5. Introduce a customer rewards program | 5. Introduce a customer service feedback system |
| 6. Introduce a customer complaint system | 6. Introduce a customer service escalation system |
| 7. Increase quality control during manufacturing | 7. Introduce a customer service knowledge base |
| 8. Increase product warranties | 8. Introduce a customer service ticketing system |
| 9. Increase resources for product support | 9. Introduce a customer service chatbot |
| 10. Increase resources for customer communication | 10. Introduce a customer 

---
### Do all the steps in a Chaining Example

In [9]:
#r "nuget: Markdig.Signed, 0.31.0"
using Microsoft.AspNetCore.Html;
using Markdig;
using System.Text.Json;
using System.Text.Json.Serialization;

//Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";


var myResult = await kernel.RunAsync(input, skillDT["Empathize"], skillDT["Define"], skillDT["Ideate"]);

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml("### Ideate: 'Design Thinking' directions generated from customer empathy (Empathize) and problem definition (Define)\n\n" + myResult.ToString(), myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent

Installed Packages Markdig.Signed, 0.31.0

Lower-hanging fruit,Higher-hanging fruit
Implement automated customer service chatbot,Develop a customer loyalty program
Create an online knowledge base for customers,Invest in better quality control during manufacturing
Improve customer service training,Improve product design to increase durability
Introduce a customer feedback system,Expand technical support staff
Offer customer service via social media,Increase product testing to ensure reliability
Increase customer service staff,Develop a customer rewards program
Offer discounts or incentives for customer loyalty,Create an online product tutorial library
Create an online customer service portal,Develop an online customer service forum
Introduce a customer satisfaction survey,Invest in customer service analytics software
